# GENESIS Level 0 Training

Train the Level 0 (Machine Code Patterns) model using Google Colab's free GPU.

## Setup
1. Go to **Runtime → Change runtime type → GPU (T4)**
2. Run all cells in order (Ctrl+F9)

In [ ]:
# Step 1: Check GPU availability
!nvidia-smi --query-gpu=name,memory.total --format=csv

In [ ]:
# Step 2: Clone the repository
import os

if os.path.exists('genesis'):
    !rm -rf genesis

!git clone https://github.com/0xMayoor/genesis.git
os.chdir('genesis')
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Step 3: Install dependencies
!pip install -q torch transformers peft accelerate capstone hypothesis

In [ ]:
# Step 4: Install GENESIS package
!pip install -e . --force-reinstall --no-deps -q
print("Installation complete!")

In [ ]:
# Step 5: Verify installation
import sys
import os

cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.insert(0, cwd)

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

from core.training import TrainingConfig, ModelConfig
from levels.level0_machine import Level0Module
print("\n✅ All imports working!")

In [ ]:
# Step 6: Check dataset
from pathlib import Path

dataset_path = Path("genesis_datasets/level0/train.jsonl")
if dataset_path.exists():
    line_count = sum(1 for _ in open(dataset_path))
    print(f"✅ Dataset exists: {dataset_path}")
    print(f"   Samples: {line_count}")
else:
    print("❌ Dataset not found!")

In [ ]:
# Step 7: Configure training
from pathlib import Path
from core.training import TrainingConfig, ModelConfig

config = TrainingConfig(
    output_dir=Path("models/level0"),
    model=ModelConfig(
        model_name="distilgpt2",
        use_lora=True,
        lora_r=8,
        lora_target_modules=["c_attn", "c_proj"],  # GPT-2 attention modules
        max_length=256,
    ),
    batch_size=8,
    num_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=50,
    save_steps=500,
)

print("Training Configuration:")
print(f"  Model: {config.model.model_name}")
print(f"  LoRA rank: {config.model.lora_r}")
print(f"  LoRA targets: {config.model.lora_target_modules}")
print(f"  Batch size: {config.batch_size}")
print(f"  Epochs: {config.num_epochs}")
print(f"  FP16: {config.fp16}")

print("\n✅ Configuration ready!")

In [ ]:
# Step 8: Train the model!
from core.training import train_level0

print("Starting training...")
print("This will take ~10-30 minutes on a T4 GPU.")
print("="*50)

metrics = train_level0(config, dataset_path)

print("\n" + "="*50)
print("🎉 Training Complete!")
print("="*50)

In [ ]:
# Step 9: Check results
print("Training Metrics:")
print(f"  Accuracy: {metrics.accuracy:.2%}")
print(f"  Adversarial Refusal Rate: {metrics.adversarial_refusal_rate:.2%}")
print(f"  False Positive Rate: {metrics.false_positive_rate:.2%}")
print(f"  Total Samples: {metrics.total_samples}")

print("\nGate Requirements:")
passes, failures = metrics.meets_gate_requirements()

if passes:
    print("✅ Model PASSES all gate requirements!")
    print("🚀 Level 0 complete. Ready for Level 1 (Assembly).")
else:
    print("❌ Model FAILS gate requirements:")
    for f in failures:
        print(f"  - {f}")

In [ ]:
# Step 10: Download trained model
!zip -r level0_model.zip models/level0/

from google.colab import files
files.download('level0_model.zip')

print("\n📦 Model downloaded!")